In [3]:
import json

from os import listdir
from os.path import isfile, join, isdir

countries_root = 'json_store/by_country/'

country_folders = [f for f in listdir(countries_root) if not isdir(f)]
print country_folders

#calculates amount of people who immigrated into a country from 
#a certain place within the month 
def immigrated_in_month_from(total_year, total_from_year, total_month):
    percent_from = total_from_year/float(total_year)
    return int(round(total_month*percent_from, 0))
    


# with open('json_store/monthly_by_country.json') as data_file:    
#     monthly_data = json.load(data_file)

['ITA', 'SPA', 'MTA', 'GRE']


In [4]:
all_data = {}

for country_folder in country_folders:
    country_obj = {}
    
    with open('{0}{1}/monthly{1}.json'.format(countries_root, country_folder)) as data_file: 
        data_f = json.load(data_file)
        prevYear = data_f[0]["year"]
        all_months = {}
        monthly = {}
        yearly = {}
        for data in data_f:
            yearly[data["year"]] = yearly.get(data["year"], 0) + data["value"]
            if data["year"] == prevYear:
                all_months[data["month"]] = data["value"]
            else:
                monthly[prevYear] = all_months
                all_months = {}
                all_months[data["month"]] = data["value"]
            prevYear = data["year"]
        monthly[prevYear] = all_months
        all_months[data["month"]] = data["value"]
        country_obj["yearly"] = yearly
        country_obj["monthly"] = monthly
        
        
        
    origin_files = [f for f in listdir(countries_root+country_folder) if 'from' in f]
    origins = {}
    for origin_file in origin_files:
        with open('{0}{1}/{2}'.format(countries_root, country_folder, origin_file)) as data_file:
                data = json.load(data_file)[0]
                origins[data["country_of_origin"]] = data["value"]
    country_obj["origins"] = origins
    
    
    all_data[country_folder] = country_obj
print all_data

with open('json_store/all_data.json', 'w') as outfile:
    json.dump(all_data, outfile)

{'SPA': {'yearly': {2016: 903, 2014: 4348, 2015: 4408}, 'monthly': {2016: {1: 447, 2: 180, 3: 276, 4: 0, 5: 0, 6: 0}, 2014: {1: 144, 2: 33, 3: 232, 4: 148, 5: 325, 6: 246, 7: 264, 8: 1705, 9: 380, 10: 341, 11: 211, 12: 319}, 2015: {1: 241, 2: 44, 3: 267, 4: 240, 5: 438, 6: 377, 7: 346, 8: 385, 9: 459, 10: 795, 11: 508, 12: 308}}, 'origins': {u'MOZ': 25, u'UGA': 4, u'MLI': 28, u'GNB': 20, u'SEN': 3, u'CMR': 135}}, 'MTA': {'yearly': {2016: 0, 2014: 568, 2015: 105}, 'monthly': {2016: {1: 0, 2: 0, 3: 0, 4: 0}, 2014: {1: 0, 2: 0, 3: 91, 4: 0, 5: 0, 6: 136, 7: 81, 8: 257, 9: 3, 10: 0, 11: 0, 12: 0}, 2015: {1: 87, 2: 0, 3: 0, 4: 5, 5: 0, 6: 2, 7: 2, 8: 2, 9: 7, 10: 0, 11: 0, 12: 0}}, 'origins': {}}, 'GRE': {'yearly': {2016: 156782, 2014: 41038, 2015: 856723}, 'monthly': {2016: {1: 67415, 2: 57066, 3: 26971, 4: 3650, 5: 1465, 6: 215}, 2014: {1: 955, 2: 1001, 3: 1501, 4: 1257, 5: 1703, 6: 3198, 7: 3927, 8: 6742, 9: 7454, 10: 7432, 11: 3812, 12: 2056}, 2015: {1: 1694, 2: 2873, 3: 7874, 4: 13556,

In [5]:
processed_data = {}
with open('json_store/all_data.json') as data_file: 
    data = json.load(data_file)
    proc_country_data = {}
    for country_code in country_folders:
        country_data = data[country_code]
        origin_data = country_data["origins"]
        years_data = {}
        for i in range(0, 3):
            year = unicode(i + 2014)
            year_data = {}
            yearsum = 0;
            for year_key, year_val in country_data["yearly"].iteritems():
                yearsum += year_val
            months_data = {}
            for month_key, month_val in sorted(country_data["monthly"][year].iteritems()):
                origin_data = {}
                for origin, origin_val in country_data["origins"].iteritems():
                    origin_data[str(origin)] = immigrated_in_month_from(yearsum, origin_val, month_val)
                months_data[int(month_key)] = origin_data
            years_data[int(year)] = months_data
        proc_country_data[country_code] = years_data
processed_data = proc_country_data
print processed_data

with open('json_store/processed_data.json', 'w') as outfile:
    json.dump(processed_data, outfile)

{'SPA': {2016: {1: {'MOZ': 1, 'UGA': 0, 'MLI': 1, 'GNB': 1, 'SEN': 0, 'CMR': 6}, 2: {'MOZ': 0, 'UGA': 0, 'MLI': 1, 'GNB': 0, 'SEN': 0, 'CMR': 3}, 3: {'MOZ': 1, 'UGA': 0, 'MLI': 1, 'GNB': 1, 'SEN': 0, 'CMR': 4}, 4: {'MOZ': 0, 'UGA': 0, 'MLI': 0, 'GNB': 0, 'SEN': 0, 'CMR': 0}, 5: {'MOZ': 0, 'UGA': 0, 'MLI': 0, 'GNB': 0, 'SEN': 0, 'CMR': 0}, 6: {'MOZ': 0, 'UGA': 0, 'MLI': 0, 'GNB': 0, 'SEN': 0, 'CMR': 0}}, 2014: {1: {'MOZ': 0, 'UGA': 0, 'MLI': 0, 'GNB': 0, 'SEN': 0, 'CMR': 2}, 2: {'MOZ': 0, 'UGA': 0, 'MLI': 0, 'GNB': 0, 'SEN': 0, 'CMR': 0}, 3: {'MOZ': 1, 'UGA': 0, 'MLI': 1, 'GNB': 0, 'SEN': 0, 'CMR': 3}, 4: {'MOZ': 0, 'UGA': 0, 'MLI': 0, 'GNB': 0, 'SEN': 0, 'CMR': 2}, 5: {'MOZ': 1, 'UGA': 0, 'MLI': 1, 'GNB': 1, 'SEN': 0, 'CMR': 5}, 6: {'MOZ': 1, 'UGA': 0, 'MLI': 1, 'GNB': 1, 'SEN': 0, 'CMR': 3}, 7: {'MOZ': 1, 'UGA': 0, 'MLI': 1, 'GNB': 1, 'SEN': 0, 'CMR': 4}, 8: {'MOZ': 4, 'UGA': 1, 'MLI': 5, 'GNB': 4, 'SEN': 1, 'CMR': 24}, 9: {'MOZ': 1, 'UGA': 0, 'MLI': 1, 'GNB': 1, 'SEN': 0, 'CMR': 5}, 